In [63]:
from transformer_srl import dataset_readers, models, predictors
from collections import OrderedDict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
stopwords = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
predictor  = predictors.SrlTransformersPredictor.from_path("/home/jessica/Downloads/srl_bert_base_conll2012.tar.gz", "transformer_srl", cuda_device=-1)  

In [75]:
def get_event_chain(predictor, sentence):
    srl_output = predictor.predict(sentence=sentence)
    sentence_tokens = srl_output['words']
    
    event_chain = OrderedDict()
    #print(srl_output)
    for count, verb in enumerate(srl_output['verbs']):        
        tags = verb['tags']
        tag_name = f"{count}_{verb['verb']}"
        event_chain[tag_name] = list()
        for index, tag in enumerate(tags):
            if 'ARG' in tag:
                event_chain[tag_name].append(f"{sentence_tokens[index]}/{tag}")
                
    return event_chain

In [76]:
def clean_data(text):
    tokens_without_sw = [word for word in text.split() if not word in stopwords]
    new_sent = ' '.join(tokens_without_sw)
    return new_sent
    

In [77]:
test_sentence = "Mary and Jacob married. Mary had a baby and took care of it. However, Jacob got a job."
get_event_chain(predictor, test_sentence)

OrderedDict([('0_married', ['Mary/B-ARG1', 'and/I-ARG1', 'Jacob/I-ARG1']),
             ('1_had', ['Mary/B-ARG0', 'a/B-ARG1', 'baby/I-ARG1']),
             ('2_took',
              ['Mary/B-ARG0', 'care/B-ARG1', 'of/B-ARG2', 'it/I-ARG2']),
             ('3_got',
              ['However/B-ARGM-DIS',
               'Jacob/B-ARG0',
               'a/B-ARG1',
               'job/I-ARG1'])])

In [81]:
test_sentence = "Mary, a beautiful young lady, teaches at a high school. She loves her job. While on a trip, she meet an handsome young man who is a doctor. They fall in love and get married. Eventually, Mary leaves her job to take care of the house which makes her very sad and depressed "
get_event_chain(predictor, clean_data(test_sentence))

OrderedDict([('0_teaches',
              ['Mary/B-ARG0',
               ',/I-ARG0',
               'beautiful/I-ARG0',
               'young/I-ARG0',
               'lady/I-ARG0',
               'high/B-ARG1',
               'school/I-ARG1']),
             ('1_loves', ['She/B-ARG0', 'job/B-ARG1']),
             ('2_meet',
              ['While/B-ARGM-TMP',
               'trip/I-ARGM-TMP',
               'handsome/B-ARG1',
               'young/I-ARG1',
               'man/I-ARG1',
               'doctor/I-ARG1']),
             ('3_fall', ['They/B-ARG1', 'love/B-ARG2']),
             ('4_get', ['They/B-ARG1', 'married/B-ARG2']),
             ('5_leaves',
              ['Eventually/B-ARGM-TMP',
               'Mary/B-ARG0',
               'job/B-ARG1',
               'take/I-ARG1']),
             ('6_take', ['Mary/B-ARG0', 'care/B-ARG2', 'house/I-ARG2']),
             ('7_makes', ['Mary/B-ARG0', 'sad/B-ARG1', 'depressed/I-ARG1'])])

In [78]:
movie_plot = """
Kabir Rajdheer Singh, a house surgeon at the Delhi Institute of Medical Sciences, suffers from severe anger management problems that gain him the reputation of a bully.\n
After a brawl, Kabir is asked to either apologise or leave; he chooses to leave but stays upon seeing a new student, Preeti Sikka.\n
Later, Kabir and his friends announce that he has exclusively claimed Preeti. \n
Initially timid, she adjusts to his overbearing attitude and eventually reciprocates his feelings, developing an intimate relationship. \n
Kabir graduates with an MBBS degree and moves to Mussoorie for post-graduation. \n
Despite the distance, their relationship strengthens over the next few years. \n
After Preeti graduates, he meets her conservative parents, but her father Harpal catches them kissing and throws him out. \n
Harpal continues to oppose their relationship, despite Kabir's attempts to explain their love. \n
Enraged, Kabir asks Preeti to choose between him and her family within the next six hours, or he will end their relationship. \n
Preeti fails to reach back to him in time; feeling abandoned, Kabir injects himself with morphine and remains unconscious for the next two days. \n
On gaining consciousness, he learns that Preeti is being forced into an arranged marriage and gate crashes the wedding party; Harpal has him beaten and arrested. \n
After Kabir is released, his father, Rajdheer, ostracises him from the family for his antics. \n
With his friend Shiva’s help, Kabir finds a rented flat and joins a private hospital as a surgeon. \n
To cope with his emotions, he abuses drugs and alcohol, attempts one-night stands, has casual relationships, buys a pet dog — none of which are successful. \n
Within months, he becomes a successful surgeon and a high-functioning alcoholic both respected and feared by the hospital staff. \n
His self-destructive behaviour and obsession worry his friends. \n
While hungover on a day-off, Kabir is called in to perform an emergency surgery for which he unwillingly agrees. \n"""

In [79]:
for sentence in movie_plot.split('\n'):
    if len(sentence)>0:
        print(get_event_chain(predictor, clean_data(sentence)), "\n\n")

OrderedDict([('0_suffers', ['Kabir/B-ARG0', 'Rajdheer/I-ARG0', 'Singh/I-ARG0', ',/I-ARG0', 'house/I-ARG0', 'surgeon/I-ARG0', 'Delhi/I-ARG0', 'Institute/I-ARG0', 'Medical/I-ARG0', 'Sciences/I-ARG0', ',/I-ARG0', 'severe/B-ARG1', 'anger/I-ARG1', 'management/I-ARG1', 'problems/I-ARG1', 'gain/I-ARG1', 'reputation/I-ARG1', 'bully/I-ARG1']), ('1_gain', ['Kabir/B-ARG0', 'Rajdheer/I-ARG0', 'Singh/I-ARG0', ',/I-ARG0', 'house/I-ARG0', 'surgeon/I-ARG0', 'Delhi/I-ARG0', 'Institute/I-ARG0', 'Medical/I-ARG0', 'Sciences/I-ARG0', 'reputation/B-ARG1', 'bully/I-ARG1'])]) 


OrderedDict([('0_asked', ['After/B-ARGM-TMP', 'brawl/I-ARGM-TMP', 'Kabir/B-ARG0', 'either/B-ARG1', 'apologise/I-ARG1', 'leave/I-ARG1']), ('1_leave', []), ('2_seeing', ['new/B-ARG1', 'student/I-ARG1', ',/I-ARG1', 'Preeti/I-ARG1', 'Sikka/I-ARG1'])]) 


OrderedDict([('0_announce', ['Later/B-ARGM-TMP', 'Kabir/B-ARG0', 'friends/I-ARG0', 'exclusively/B-ARG1']), ('1_claimed', ['Kabir/B-ARG0', 'friends/I-ARG0', 'exclusively/B-ARGM-MNR', 'Pree